In [26]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

#library for connecting to the geodatabase
from sqlalchemy import create_engine, text
from geoalchemy2 import Geometry
import seaborn as sns

In [3]:
root_directory = r"C:\Users\abhimanya.achara\Downloads\e56209c9a5c393b9e0af53045dcce1ac81c438e4"

In [4]:
def load_csvs_from_subfolders(root_folder):
    # List to hold each DataFrame
    df_list = []

    # Walk through the directory structure
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for file in filenames:
            if file.endswith('.csv'):
                file_path = os.path.join(dirpath, file)
                df = pd.read_csv(file_path)
                df_list.append(df)

    # Combine all dataframes
    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df

In [5]:
final_dataframe = load_csvs_from_subfolders(root_directory)

# Optional: print shape or save to CSV
final_dataframe.tail()


,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
330758,586c2acfeec476872d2006deadd21dfe2b20f00c8c5fe6...,2025-02,West Yorkshire Police,West Yorkshire Police,NaN,NaN,No Location,NaN,NaN,Other crime,Under investigation,NaN
330759,29f3cbea283183bab9d2b5b9b66d5a51fa2f68eb565746...,2025-02,West Yorkshire Police,West Yorkshire Police,NaN,NaN,No Location,NaN,NaN,Other crime,Under investigation,NaN
330760,86a65f347bc12d7c381440f36941124bfe552173e0ff2d...,2025-02,West Yorkshire Police,West Yorkshire Police,NaN,NaN,No Location,NaN,NaN,Other crime,Under investigation,NaN
330761,2a3172cdc2706403be0d6b20339afc5b8912c1d5137050...,2025-02,West Yorkshire Police,West Yorkshire Police,NaN,NaN,No Location,NaN,NaN,Other crime,Under investigation,NaN
330762,20661627214917519f942457ed3f1a22e9f00af0a8fce4...,2025-02,West Yorkshire Police,West Yorkshire Police,NaN,NaN,No Location,NaN,NaN,Other crime,Under investigation,NaN


In [6]:
#Drop rows with no locations
final_dataframe = final_dataframe[final_dataframe['Location'] != 'No Location']
final_dataframe.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,NaN,2024-02,West Yorkshire Police,West Yorkshire Police,-1.472961,53.599861,On or near Railway Walk,E01007439,Barnsley 001D,Anti-social behaviour,NaN,NaN
1,NaN,2024-02,West Yorkshire Police,West Yorkshire Police,-1.502826,53.599297,On or near Warren Lane,E01007356,Barnsley 004A,Anti-social behaviour,NaN,NaN
2,1a044ad4d0b924c325224012ab9c9feb3a627781cc0fe8...,2024-02,West Yorkshire Police,West Yorkshire Police,-1.356632,53.589279,On or near Common Road,E01007328,Barnsley 006A,Other theft,Investigation complete; no suspect identified,NaN
3,ebc988acc4ae545dd9630f8110529b2cd93f31a4c78172...,2024-02,West Yorkshire Police,West Yorkshire Police,-1.721901,53.559011,On or near Park/Open Space,E01007426,Barnsley 027D,Violence and sexual offences,Status update unavailable,NaN
4,c9fb15796ddb7976e1e6d15a06df03bb13a785061b784b...,2024-02,West Yorkshire Police,West Yorkshire Police,-1.906708,53.939023,On or near Turner Lane,E01010646,Bradford 001A,Drugs,Unable to prosecute suspect,NaN


In [7]:
#Create dataframe of number of crimes per location
final_simplified_dataframe = final_dataframe.groupby(['Longitude','Latitude','Location']).size().reset_index(name='count')
final_simplified_dataframe.head()

,Longitude,Latitude,Location,count
0,-2.734806,53.459170,On or near Swift Street,1
1,-2.155415,53.733818,On or near Ivy Place,3
2,-2.154682,53.732381,On or near Parking Area,9
3,-2.153104,53.732113,On or near Lennox Road,35
4,-2.153063,53.733399,On or near Station Parade,1


In [8]:
#Create dataframe of number of crimes per location per type
final_crime_type_by_location_dataframe = final_dataframe.groupby(['Longitude','Latitude','Location','Crime type']).size().reset_index(name='count')
final_crime_type_by_location_dataframe.head()

,Longitude,Latitude,Location,Crime type,count
0,-2.734806,53.459170,On or near Swift Street,Vehicle crime,1
1,-2.155415,53.733818,On or near Ivy Place,Public order,1
2,-2.155415,53.733818,On or near Ivy Place,Vehicle crime,1
3,-2.155415,53.733818,On or near Ivy Place,Violence and sexual offences,1
4,-2.154682,53.732381,On or near Parking Area,Anti-social behaviour,4


In [14]:
# Create geometry column: Point(longitude, latitude)
final_simplified_dataframe['geometry'] = final_simplified_dataframe.apply(
    lambda row: Point(row['Longitude'], row['Latitude']), axis=1
)

# Convert to GeoDataFrame
all_crime_df = gpd.GeoDataFrame(final_simplified_dataframe, geometry='geometry')

# Set the coordinate reference system (CRS) to WGS84 (EPSG:4326)
all_crime_df.set_crs(epsg=4326, inplace=True)

all_crime_df['id'] = range(1, len(all_crime_df) + 1)

# Optional: preview
all_crime_df.head()

,Longitude,Latitude,Location,count,geometry,id
0,-2.734806,53.459170,On or near Swift Street,1,POINT (-2.73481 53.45917),1
1,-2.155415,53.733818,On or near Ivy Place,3,POINT (-2.15542 53.73382),2
2,-2.154682,53.732381,On or near Parking Area,9,POINT (-2.15468 53.73238),3
3,-2.153104,53.732113,On or near Lennox Road,35,POINT (-2.15310 53.73211),4
4,-2.153063,53.733399,On or near Station Parade,1,POINT (-2.15306 53.73340),5


In [13]:
# Create geometry column: Point(longitude, latitude)
final_crime_type_by_location_dataframe['geometry'] = final_crime_type_by_location_dataframe.apply(
    lambda row: Point(row['Longitude'], row['Latitude']), axis=1
)

# Convert to GeoDataFrame
detailed_crime_df = gpd.GeoDataFrame(final_crime_type_by_location_dataframe, geometry='geometry')

# Set the coordinate reference system (CRS) to WGS84 (EPSG:4326)
detailed_crime_df.set_crs(epsg=4326, inplace=True)

detailed_crime_df['id'] = range(1, len(detailed_crime_df) + 1)

# Optional: preview
detailed_crime_df.head()

,Longitude,Latitude,Location,Crime type,count,geometry,id
0,-2.734806,53.459170,On or near Swift Street,Vehicle crime,1,POINT (-2.73481 53.45917),1
1,-2.155415,53.733818,On or near Ivy Place,Public order,1,POINT (-2.15542 53.73382),2
2,-2.155415,53.733818,On or near Ivy Place,Vehicle crime,1,POINT (-2.15542 53.73382),3
3,-2.155415,53.733818,On or near Ivy Place,Violence and sexual offences,1,POINT (-2.15542 53.73382),4
4,-2.154682,53.732381,On or near Parking Area,Anti-social behaviour,4,POINT (-2.15468 53.73238),5


In [22]:
# Define database connection parameters
db_host = "PRIORPSRV03"
db_name = "gis"
db_port = "5432"
db_schema = "university_of_leeds"
table_name = "polikeuk_detailed_crime_locations"  # Desired table name
primary_key_column = "id"  # Define the primary key column (change based on your dataset)
geometry_column = "geometry"  # Default geometry column
# Create the database connection string (Windows Authentication - Trusted Connection)
conn_str = f"postgresql+psycopg2://@{db_host}:{db_port}/{db_name}?sslmode=disable"

# Create a SQLAlchemy engine
engine = create_engine(conn_str)

In [23]:
layer_to_load = detailed_crime_df

In [24]:
# Ensure the GeoDataFrame has a valid CRS before writing
if layer_to_load.crs is None:
    print("Warning: GeoDataFrame has no CRS. Setting default to EPSG:27700 (British National Grid).")
    layer_to_load.set_crs(epsg=4236, inplace=True)

In [25]:
# Automatically detect geometry type from GeoDataFrame
geom_type = layer_to_load.geom_type.unique()[0].upper()
# Publish the GeoDataFrame to PostGIS
layer_to_load.to_postgis(
    name=table_name,
    con=engine,
    schema=db_schema,
    if_exists="replace",
    index=False,
    dtype = {'geometry': geom_type}
)

print(f"Data successfully uploaded to PostGIS: {db_schema}.{table_name}")

# Connect to the database to modify table structure
with engine.connect() as conn:
    # Set Primary Key (if it doesn't exist already)
    alter_pk_query = text(f"""
        ALTER TABLE {db_schema}.{table_name}
        ADD CONSTRAINT {table_name}_pkey PRIMARY KEY ({primary_key_column});
    """)
    
    # Create Spatial Index
    create_spatial_index_query = text(f"""
        CREATE INDEX {table_name}_geom_idx
        ON {db_schema}.{table_name}
        USING GIST ({geometry_column});
    """)

    try:
        conn.execute(alter_pk_query)  # Add Primary Key
        print(f"Primary key set on column: {primary_key_column}")
    except Exception as e:
        print(f"Could not set primary key. It may already exist. Error: {e}")

    try:
        conn.execute(create_spatial_index_query)  # Add Spatial Index
        print(f"Spatial index created for geometry column: {geometry_column}")
    except Exception as e:
        print(f"Could not create spatial index. It may already exist. Error: {e}")

print(f"GeoDataFrame successfully published to PostGIS with Primary Key and Spatial Index: {db_schema}.{table_name}")

Data successfully uploaded to PostGIS: university_of_leeds.polikeuk_detailed_crime_locations
Primary key set on column: id
Spatial index created for geometry column: geometry
GeoDataFrame successfully published to PostGIS with Primary Key and Spatial Index: university_of_leeds.polikeuk_detailed_crime_locations
